In [1]:
from binn import BINNClassifier, Network, SuperLogger, BINNExplainer, BINN
import pandas as pd

input_data = pd.read_csv("../data/test_qm.csv")
translation = pd.read_csv("../data/translation.tsv", sep="\t")
pathways = pd.read_csv("../data/pathways.tsv", sep="\t")

# Create the model
network = Network(
    input_data=input_data,
    pathways=pathways,
    mapping=translation,
    verbose=False
)

binn = BINN(
    pathways=network,
    n_layers=4,
    dropout=0.2,
    validate=False,
    residual=True
)

/home/erikh/BINN/BINN/test-venv/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Base case reached
Final number of unique connections in pathway:  1003
Number of copies made for 4 layers: 26


In [2]:
from UtilForExamples import fit_protein_matrix_to_network_input, generate_data
import torch
from pytorch_lightning import Trainer

design_matrix = pd.read_csv('../data/design_matrix.tsv', sep="\t")
protein_matrix = pd.read_csv('../data/test_qm.csv')

protein_matrix = fit_protein_matrix_to_network_input(
    protein_matrix, features=network.inputs)

X, y = generate_data(protein_matrix, design_matrix=design_matrix)

dataloader = torch.utils.data.DataLoader(dataset=torch.utils.data.TensorDataset(torch.Tensor(X), torch.LongTensor(y)),
                                            batch_size=8,
                                            num_workers=12,
                                            shuffle=True)
trainer = Trainer(max_epochs=5)
trainer.fit(binn, dataloader)

GPU available: True (cuda), used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
GPU available but not used. Set `accelerator` and `devices` using `Trainer(accelerator='gpu', devices=1)`.
You defined a `validation_step` but have no `val_dataloader`. Skipping val loop.

  | Name   | Type             | Params
--------------------------------------------
0 | layers | Sequential       | 365 K 
1 | loss   | CrossEntropyLoss | 0     
--------------------------------------------
365 K     Trainable params
0         Non-trainable params
365 K     Total params
1.464     Total estimated model params size (MB)
The number of training batches (25) is smaller than the logging interval Trainer(log_every_n_steps=50). Set a lower value for log_every_n_steps if you want to see logs for the training epoch.


Epoch 4: 100%|██████████| 25/25 [00:00<00:00, 36.87it/s, loss=0.653, v_num=11, train_loss=0.653, train_acc=0.756]

`Trainer.fit` stopped: `max_epochs=5` reached.


Epoch 4: 100%|██████████| 25/25 [00:00<00:00, 33.84it/s, loss=0.653, v_num=11, train_loss=0.653, train_acc=0.756]


In [3]:
from binn import BINNExplainer

test_data = torch.Tensor(X)
background_data = torch.Tensor(X)

explainer = BINNExplainer(binn)

In [3]:
importance_df = explainer.explain(test_data, background_data)
importance_df

Using a non-full backward hook when the forward contains multiple autograd Nodes is deprecated and will be removed in future versions. This hook will be missing some grad_input. Please use register_full_backward_hook to get the documented behavior.


In [6]:
from binn import complete_shap_sankey

complete_shap_sankey(importance_df, savename='../img/test.png')


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/

In [9]:
from binn import ImportanceNetwork

IG = ImportanceNetwork(importance_df)

query_node = 'P02766'

IG.generate_sankey(query_node, upstream=False, savename='../img/sankey.png')

/home/erikh/BINN/BINN/binn/plot.py:29: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/home/erikh/BINN/BINN/binn/plot.py:30: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/home/erikh/BINN/BINN/binn/plot.py:21: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [4]:
average_importance_df = explainer.explain_average(test_data = test_data, 
                                                  background_data = background_data, 
                                                  nr_iterations= 2, 
                                                  trainer=trainer,
                                                  dataloader= dataloader)
average_importance_df

nn.init.xavier_uniform is now deprecated in favor of nn.init.xavier_uniform_.
Checkpoint directory /home/erikh/BINN/BINN/notebooks/lightning_logs/version_11/checkpoints exists and is not empty.

  | Name   | Type             | Params
--------------------------------------------
0 | layers | Sequential       | 365 K 
1 | loss   | CrossEntropyLoss | 0     
--------------------------------------------
365 K     Trainable params
0         Non-trainable params
365 K     Total params
1.464     Total estimated model params size (MB)
`Trainer.fit` stopped: `max_epochs=5` reached.
Using a non-full backward hook when the forward contains multiple autograd Nodes is deprecated and will be removed in future versions. This hook will be missing some grad_input. Please use register_full_backward_hook to get the documented behavior.


Number of copies made for 4 layers: 26


nn.init.xavier_uniform is now deprecated in favor of nn.init.xavier_uniform_.
You defined a `validation_step` but have no `val_dataloader`. Skipping val loop.
Checkpoint directory /home/erikh/BINN/BINN/notebooks/lightning_logs/version_11/checkpoints exists and is not empty.

  | Name   | Type             | Params
--------------------------------------------
0 | layers | Sequential       | 365 K 
1 | loss   | CrossEntropyLoss | 0     
--------------------------------------------
365 K     Trainable params
0         Non-trainable params
365 K     Total params
1.464     Total estimated model params size (MB)
`Trainer.fit` stopped: `max_epochs=5` reached.
Using a non-full backward hook when the forward contains multiple autograd Nodes is deprecated and will be removed in future versions. This hook will be missing some grad_input. Please use register_full_backward_hook to get the documented behavior.


Number of copies made for 4 layers: 26


,source,target,type,source layer,target layer,value_0,value_1,value_mean,values_std,value
0,A0M8Q6_0,R-HSA-166663_1,0,0,1,1.321620e-04,2.662810e-04,1.992215e-04,6.705949e-05,1.992215e-04
1,A0M8Q6_0,R-HSA-166663_1,1,0,1,4.747831e-04,3.155029e-04,3.951430e-04,7.964009e-05,3.951430e-04
2,A0M8Q6_0,R-HSA-977606_1,0,0,1,1.321620e-04,2.662810e-04,1.992215e-04,6.705949e-05,1.992215e-04
3,A0M8Q6_0,R-HSA-977606_1,1,0,1,4.747831e-04,3.155029e-04,3.951430e-04,7.964009e-05,3.951430e-04
4,A0M8Q6_0,R-HSA-2029481_1,0,0,1,1.321620e-04,2.662810e-04,1.992215e-04,6.705949e-05,1.992215e-04
...,...,...,...,...,...,...,...,...,...,...
6901,R-HSA-1852241_4,root_5,1,4,5,6.102887e-08,3.821710e-08,4.962298e-08,1.140588e-08,4.962298e-08
6902,R-HSA-112316_4,root_5,0,4,5,7.387129e-10,8.608691e-11,4.123999e-10,3.263130e-10,4.123999e-10
6903,R-HSA-112316_4,root_5,1,4,5,4.207380e-10,4.006940e-10,4.107160e-10,1.002199e-11,4.107160e-10
6904,R-HSA-4839726_4,root_5,0,4,5,1.396211e-08,2.923173e-08,2.159692e-08,7.634812e-09,2.159692e-08
